# Day 4: Printing Department ---
```
You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really big print jobs).

Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline.

"Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around."

If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall.

The rolls of paper (@) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located.

For example:

..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
The forklifts can only access a roll of paper if there are fewer than four rolls of paper in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria.

In this example, there are 13 rolls of paper that can be accessed by a forklift (marked with x):

..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
Consider your complete diagram of the paper roll locations. How many rolls of paper can be accessed by a forklift?
```

In [1]:
import scipy
import numpy as np
import sys 
sys.path.append("..")

from common.utils import Day, read_as_character_array

test_path = "test.txt"
data_path = "data.txt"


In [2]:
"""
IDEAS:
turn "@" into 1 and "." into 0:
    Kernel convolution with python package
    kernel = [
        1, 1, 1,
        1, 0, 1,
        1, 1, 1,
        ]
        conv_diagram =scipy.convolve(binary_diagram, kernel)
        result = sum(conv_diagram >=4)

"""
class Day4(Day):
    def __init__(self, path):
        super().__init__(path)
        self.diagram = read_as_character_array(path)

    def solve_part_1(self):
        """
        Create a binary representation of the diagram
        "@" turn into 1, "." get into 0.
        the number of neigbours can be counted with the
        convolution kernel and binary diagram.
        """
        kernel = np.array([
            1, 1, 1,
            1, 0, 1,
            1, 1, 1,
        ]).reshape(3,3)

        self.binary_diagram = np.where(self.diagram == "@", 1, 0)
        self.conv_diagram = scipy.ndimage.convolve(
            self.binary_diagram, kernel, mode = "constant", cval = 0,
        )

        # We only care about neighbours of rolls, not empty spaces
        roll_neighbours = np.multiply(self.binary_diagram, self.conv_diagram)

        final_mask = np.where((roll_neighbours < 4) & (self.binary_diagram == 1), 1, 0)

        return np.sum(final_mask)

day4 = Day4(test_path)
day4.solve_part_1()

np.int64(13)

In [3]:
day4 = Day4(data_path)
day4.solve_part_1()

np.int64(1569)

# Part Two ---
```
Now, the Elves just need help accessing as much of the paper as they can.

Once a roll of paper can be accessed by a forklift, it can be removed. Once a roll of paper is removed, the forklifts might be able to access more rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process?

Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted @ to indicate that a roll of paper is about to be removed, and using x to indicate that a roll of paper was just removed:

Initial state:
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.

Remove 13 rolls of paper:
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
.@@.x.x.@x
x@@@@...@@
x.@@@@..x.
.@.@@@@.x.
.x@@@@@@.x
.x.@.@.@@@
..@@@.@@@@
.x@@@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x@.....x.
.@@@@...xx
..@@@@....
.x.@@@@...
..@@@@@@..
...@.@.@@x
..@@@.@@@@
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x@@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@@.
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
..@@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
...@@.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x@.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
Stop once no more rolls of paper are accessible by a forklift. In this example, a total of 43 rolls of paper can be removed.

Start with your original diagram. How many rolls of paper in total can be removed by the Elves and their forklifts?
```

In [25]:
"""
first lets try to run part 1 as many times as needed untill no rolls are removed
"""

class Day4(Day):
    def __init__(self, path):
        super().__init__(path)
        self.diagram = read_as_character_array(path)
        pass


    def remove_rolls(self):
        """
        Modify part 1 to update the diagram and return the number of rolls
        """
        kernel = np.array([
            1, 1, 1,
            1, 0, 1,
            1, 1, 1,
        ]).reshape(3,3)

        self.binary_diagram = np.where(self.diagram == "@", 1, 0)
        self.conv_diagram = scipy.ndimage.convolve(
            self.binary_diagram, kernel, mode = "constant", cval = 0,
        )

        # We only care about neighbours of rolls, not empty spaces
        roll_neighbours = np.multiply(self.binary_diagram, self.conv_diagram)

        final_mask = np.where((roll_neighbours < 4) & (self.binary_diagram == 1), 1, 0)

        # Update the diagram without the removed rolls
        self.diagram = np.where(final_mask == 1, ".", self.diagram)

        return np.sum(final_mask)
    

    def solve_part_2(self):
        """
        Run the method to remove rolls while keeping count
        of how many rolls are removed.

        If no rolls are removed, stop and return the sum
        """

        max_iter = 100
        i=0
        result = 0
        rolls_removed = 1
        while rolls_removed > 0:

            rolls_removed = self.remove_rolls() 
            result += rolls_removed

            i+=1
            if i > max_iter:
                print("MAX ITERATION REACHED")
                break

        return result

day4 = Day4(test_path)
print(day4.diagram)


[['.' '.' '@' '@' '.' '@' '@' '@' '@' '.']
 ['@' '@' '@' '.' '@' '.' '@' '.' '@' '@']
 ['@' '@' '@' '@' '@' '.' '@' '.' '@' '@']
 ['@' '.' '@' '@' '@' '@' '.' '.' '@' '.']
 ['@' '@' '.' '@' '@' '@' '@' '.' '@' '@']
 ['.' '@' '@' '@' '@' '@' '@' '@' '.' '@']
 ['.' '@' '.' '@' '.' '@' '.' '@' '@' '@']
 ['@' '.' '@' '@' '@' '.' '@' '@' '@' '@']
 ['.' '@' '@' '@' '@' '@' '@' '@' '@' '.']
 ['@' '.' '@' '.' '@' '@' '@' '.' '@' '.']]


In [37]:
day4.remove_rolls()
print(day4.diagram)


[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '@' '@' '.' '.' '.' '.']
 ['.' '.' '.' '@' '@' '@' '@' '.' '.' '.']
 ['.' '.' '.' '@' '@' '@' '@' '@' '.' '.']
 ['.' '.' '.' '@' '.' '@' '.' '@' '@' '.']
 ['.' '.' '.' '@' '@' '.' '@' '@' '@' '.']
 ['.' '.' '.' '@' '@' '@' '@' '@' '.' '.']
 ['.' '.' '.' '.' '@' '@' '@' '.' '.' '.']]


In [ ]:
day4 = Day4(data_path)
day4.solve_part_2()